In [1]:
import neuroglancer
import numpy as np

viewer = neuroglancer.Viewer()
viewer

http://127.0.0.1:49928/v/0220652f10f542eec8f689e1acd5f5876b0ebe9e/

# V3

In [16]:
# get a url into a nice dict
viewer_state = neuroglancer.parse_url('https://neuroglancer-demo.appspot.com/#!%7B%22dimensions%22:%7B%22x%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22y%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22z%22:%5B3.0000000000000004e-8%2C%22m%22%5D%7D%2C%22position%22:%5B5515.27392578125%2C6446.1376953125%2C1218.5%5D%2C%22crossSectionScale%22:3.7621853549999242%2C%22projectionOrientation%22:%5B-0.05179581791162491%2C-0.8017329573631287%2C0.0831851214170456%2C-0.5895944833755493%5D%2C%22projectionScale%22:4699.372698097029%2C%22layers%22:%5B%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image%22%2C%22tab%22:%22source%22%2C%22name%22:%22original-image%22%2C%22visible%22:false%7D%2C%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image_color_corrected%22%2C%22tab%22:%22source%22%2C%22name%22:%22corrected-image%22%7D%2C%7B%22type%22:%22segmentation%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/ground_truth%22%2C%22tab%22:%22source%22%2C%22selectedAlpha%22:0.63%2C%22notSelectedAlpha%22:0.14%2C%22segments%22:%5B%223208%22%2C%224901%22%2C%2213%22%2C%224965%22%2C%224651%22%2C%222282%22%2C%223189%22%2C%223758%22%2C%2215%22%2C%224027%22%2C%223228%22%2C%22444%22%2C%223207%22%2C%223224%22%2C%223710%22%5D%2C%22name%22:%22ground_truth%22%7D%5D%2C%22layout%22:%224panel%22%7D')

In [62]:
a = neuroglancer.Viewer(token='http://127.0.0.1:64691/v/b03e32f0163614d40bfce3bced2ab08789a412ab/')

In [63]:
# find viewers created with neuroglancer.Viewer()
[print(i) for i in neuroglancer.server.global_server.viewers.items()]

('1354e5fd2ab0e2c1433751933acc843b9cf7de8a', http://127.0.0.1:64691/v/1354e5fd2ab0e2c1433751933acc843b9cf7de8a/)
('6a11a08e027c4b60a51127f7da5e06906da7c97a', http://127.0.0.1:64691/v/6a11a08e027c4b60a51127f7da5e06906da7c97a/)
('b03e32f0163614d40bfce3bced2ab08789a412ab', http://127.0.0.1:64691/v/b03e32f0163614d40bfce3bced2ab08789a412ab/)
('http://127.0.0.1:64691/v/b03e32f0163614d40bfce3bced2ab08789a412ab/', http://127.0.0.1:64691/v/http://127.0.0.1:64691/v/b03e32f0163614d40bfce3bced2ab08789a412ab//)


[None, None, None, None]

'http://127.0.0.1:64691/v/http://127.0.0.1:64691/v/b03e32f0163614d40bfce3bced2ab08789a412ab//'

In [26]:
import panel as pn

# Required to render Panel objects in a Jupyter notebook
pn.extension()

class NeuroglancerViewerApp(pn.viewable.Viewer):
    def __init__(self, **params):
        super().__init__(**params)
        self.url_input = pn.widgets.TextInput(name="Neuroglancer URL", placeholder="Enter URL here...")
        self.load_button = pn.widgets.Button(name="Load", button_type="primary")
        self.load_button.on_click(self.load_viewer)
        
        self.save_to_png = pn.widgets.FileDownload(
            filename="neuroglancer.png",
            label="Download .png",
            button_type="primary",
            callback=self.take_screenshot,
        )
        
        self.iframe = pn.pane.HTML(sizing_mode='stretch_width')
        
        self.layout = pn.Column(
            self.url_input,
            pn.Row(self.load_button, self.save_to_png),
            self.iframe,
            sizing_mode='stretch_width',
        )
    
    def load_viewer(self, event):
        self.iframe.object = f'<iframe src="{self.url_input.value}" width="800" height="600"></iframe>'
    
    def take_screenshot(self):
        viewer = neuroglancer.Viewer() # TODO: instead of creating a new Viewer instance, I want to grab the existing one from the URL
        screenshot = viewer.screenshot(size=[100, 100])

        self.save_to_png.loading = True

        b = BytesIO()
        self.plot.save(b, as_png=True)
        b.seek(0)

        self.save_to_png.loading = False
        return b
        
    
    def __panel__(self):
        return self.layout

app = NeuroglancerViewerApp()
app.servable()


Column(sizing_mode='stretch_width')
    [0] TextInput(name='Neuroglancer URL', placeholder='Enter URL here...')
    [1] Row
        [0] Button(button_type='primary', name='Load')
        [1] FileDownload(button_type='primary', callback=<bound method N..., filename='neuroglancer.png', label='Download .png')
    [2] HTML(None, sizing_mode='stretch_width')

# Take a screenshot with their tools.. but of a viewer object that I did not creat with the url above

In [ ]:
c = neuroglancer.ScreenshotSaver(viewer, './')
c.capture()

# V4

In [49]:
import panel as pn
pn.extension()

class NeuroglancerViewerApp(pn.viewable.Viewer):
    def __init__(self, **params):
        super().__init__(**params)
        self.url_input = pn.widgets.TextInput(placeholder="Enter Neuroglancer URL and clock Load", width=710)
        self.url_input.param.watch(self.update_iframe, 'value')
        self.load_button = pn.widgets.Button(name="Load", button_type="primary")
        self.load_button.on_click(self.update_iframe)
        
        self.iframe = pn.pane.HTML(sizing_mode='stretch_width')
        
        # Initially load the iframe with the content from the TextInput
        self.update_iframe(None)
        
        self.layout = pn.Column(
            self.url_input,
            self.load_button,
            self.iframe,
            min_width=800,
            min_height=800,
            sizing_mode='stretch_both',
        )

    def update_iframe(self, event):
        self.iframe.object = f'<iframe src="{self.url_input.value}" width="800" height="600"></iframe>'
        
    def __panel__(self):
        return self.layout

app = NeuroglancerViewerApp()
app.layout.servable()


Column(min_height=800, min_width=800, sizing_mode='stretch_both')
    [0] TextInput(placeholder='Enter Neuroglancer U..., width=710)
    [1] Button(button_type='primary', name='Load')
    [2] HTML(str, sizing_mode='stretch_width')

In [4]:
base_state = neuroglancer.parse_url('https://neuroglancer-demo.appspot.com/#!%7B%22dimensions%22:%7B%22x%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22y%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22z%22:%5B3.0000000000000004e-8%2C%22m%22%5D%7D%2C%22position%22:%5B5029.42333984375%2C6217.5849609375%2C1182.5%5D%2C%22crossSectionScale%22:3.7621853549999242%2C%22projectionOrientation%22:%5B-0.05179581791162491%2C-0.8017329573631287%2C0.0831851214170456%2C-0.5895944833755493%5D%2C%22projectionScale%22:4699.372698097029%2C%22layers%22:%5B%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image%22%2C%22tab%22:%22source%22%2C%22name%22:%22original-image%22%7D%2C%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image_color_corrected%22%2C%22tab%22:%22source%22%2C%22name%22:%22corrected-image%22%7D%2C%7B%22type%22:%22segmentation%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/ground_truth%22%2C%22tab%22:%22source%22%2C%22selectedAlpha%22:0.63%2C%22notSelectedAlpha%22:0.14%2C%22segments%22:%5B%223208%22%2C%224901%22%2C%2213%22%2C%224965%22%2C%224651%22%2C%222282%22%2C%223189%22%2C%223758%22%2C%2215%22%2C%224027%22%2C%223228%22%2C%22444%22%2C%223207%22%2C%223224%22%2C%223710%22%5D%2C%22name%22:%22ground_truth%22%7D%5D%2C%22layout%22:%224panel%22%7D')

In [5]:
bs = neuroglancer.ViewerState(base_state)

In [7]:
bs.to_json()

{'dimensions': {'x': [6.000000000000001e-09, 'm'],
  'y': [6.000000000000001e-09, 'm'],
  'z': [3.0000000000000004e-08, 'm']},
 'position': [5029.42333984375, 6217.5849609375, 1182.5],
 'crossSectionScale': 3.7621853549999242,
 'projectionOrientation': [-0.05179581791162491,
  -0.8017329573631287,
  0.0831851214170456,
  -0.5895944833755493],
 'projectionScale': 4699.372698097029,
 'layers': [{'type': 'image',
   'source': 'precomputed://gs://neuroglancer-public-data/kasthuri2011/image',
   'tab': 'source',
   'name': 'original-image'},
  {'type': 'image',
   'source': 'precomputed://gs://neuroglancer-public-data/kasthuri2011/image_color_corrected',
   'tab': 'source',
   'name': 'corrected-image'},
  {'type': 'segmentation',
   'source': 'precomputed://gs://neuroglancer-public-data/kasthuri2011/ground_truth',
   'tab': 'source',
   'selectedAlpha': 0.63,
   'notSelectedAlpha': 0.14,
   'segments': ['3208',
    '4901',
    '13',
    '4965',
    '4651',
    '2282',
    '3189',
    '3758

# V5

In [15]:
import urllib.parse
url  = 'https://neuroglancer-demo.appspot.com/#!%7B%22dimensions%22:%7B%22x%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22y%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22z%22:%5B3.0000000000000004e-8%2C%22m%22%5D%7D%2C%22position%22:%5B5029.42333984375%2C6217.5849609375%2C1182.5%5D%2C%22crossSectionScale%22:3.7621853549999242%2C%22projectionOrientation%22:%5B-0.05179581791162491%2C-0.8017329573631287%2C0.0831851214170456%2C-0.5895944833755493%5D%2C%22projectionScale%22:4699.372698097029%2C%22layers%22:%5B%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image%22%2C%22tab%22:%22source%22%2C%22name%22:%22original-image%22%7D%2C%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image_color_corrected%22%2C%22tab%22:%22source%22%2C%22name%22:%22corrected-image%22%7D%2C%7B%22type%22:%22segmentation%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/ground_truth%22%2C%22tab%22:%22source%22%2C%22selectedAlpha%22:0.63%2C%22notSelectedAlpha%22:0.14%2C%22segments%22:%5B%223208%22%2C%224901%22%2C%2213%22%2C%224965%22%2C%224651%22%2C%222282%22%2C%223189%22%2C%223758%22%2C%2215%22%2C%224027%22%2C%223228%22%2C%22444%22%2C%223207%22%2C%223224%22%2C%223710%22%5D%2C%22name%22:%22ground_truth%22%7D%5D%2C%22layout%22:%224panel%22%7D'
result = urllib.parse.urlparse(url)
json_string = url_fragment_to_json(result)

In [16]:
result

ParseResult(scheme='https', netloc='neuroglancer-demo.appspot.com', path='/', params='', query='', fragment='!%7B%22dimensions%22:%7B%22x%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22y%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22z%22:%5B3.0000000000000004e-8%2C%22m%22%5D%7D%2C%22position%22:%5B5029.42333984375%2C6217.5849609375%2C1182.5%5D%2C%22crossSectionScale%22:3.7621853549999242%2C%22projectionOrientation%22:%5B-0.05179581791162491%2C-0.8017329573631287%2C0.0831851214170456%2C-0.5895944833755493%5D%2C%22projectionScale%22:4699.372698097029%2C%22layers%22:%5B%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image%22%2C%22tab%22:%22source%22%2C%22name%22:%22original-image%22%7D%2C%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image_color_corrected%22%2C%22tab%22:%22source%22%2C%22name%22:%22corrected-image%22%7D%2C%7B%22type%22:%22segmentation%22%2C%22source%22:%22precomputed://

In [3]:
import panel as pn
# import urllib.parse
import neuroglancer

pn.extension()

class NeuroglancerViewerApp(pn.viewable.Viewer):
    def __init__(self, **params):
        super().__init__(**params)
        self.url_input = pn.widgets.TextInput(placeholder="Enter Neuroglancer URL and click Load", width=650)
        self.load_button = pn.widgets.Button(name="Load", button_type="primary")
        self.load_button.on_click(self.update_view)
        self.load_demo_button = pn.widgets.Button(name="Demo", button_type="warning")
        self.demo_url = 'https://neuroglancer-demo.appspot.com/#!%7B%22dimensions%22:%7B%22x%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22y%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22z%22:%5B3.0000000000000004e-8%2C%22m%22%5D%7D%2C%22position%22:%5B5029.42333984375%2C6217.5849609375%2C1182.5%5D%2C%22crossSectionScale%22:3.7621853549999242%2C%22projectionOrientation%22:%5B-0.05179581791162491%2C-0.8017329573631287%2C0.0831851214170456%2C-0.5895944833755493%5D%2C%22projectionScale%22:4699.372698097029%2C%22layers%22:%5B%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image%22%2C%22tab%22:%22source%22%2C%22name%22:%22original-image%22%7D%2C%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image_color_corrected%22%2C%22tab%22:%22source%22%2C%22name%22:%22corrected-image%22%7D%2C%7B%22type%22:%22segmentation%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/ground_truth%22%2C%22tab%22:%22source%22%2C%22selectedAlpha%22:0.63%2C%22notSelectedAlpha%22:0.14%2C%22segments%22:%5B%223208%22%2C%224901%22%2C%2213%22%2C%224965%22%2C%224651%22%2C%222282%22%2C%223189%22%2C%223758%22%2C%2215%22%2C%224027%22%2C%223228%22%2C%22444%22%2C%223207%22%2C%223224%22%2C%223710%22%5D%2C%22name%22:%22ground_truth%22%7D%5D%2C%22layout%22:%224panel%22%7D'
        self.load_demo_button.on_click(self.load_demo)
        self.iframe = pn.pane.HTML(sizing_mode='stretch_width')
        self.json_pane = pn.pane.JSON({}, name='Parsed URL', height=600, width=400)

        input_layout = pn.Row(self.url_input, self.load_button, self.load_demo_button)

        self.layout = pn.Column(
                        pn.Row(input_layout),
                        pn.Row(self.iframe, self.json_pane),
                        sizing_mode='stretch_both'
        )
        
    def load_demo(self, event):
        self.url_input.value = self.demo_url
        self.load_button.clicks+=1
            
    def update_view(self, event):
        self.iframe.object = f'<iframe src="{self.url_input.value}" width="1000" height="1000"></iframe>'
        self.update_json_pane()

    def update_json_pane(self):
        try:
            parsed_url = neuroglancer.parse_url(self.url_input.value).to_json()
            # result = urllib.parse.urlparse(url)
            self.json_pane.object = parsed_url
        except Exception as e:
            self.json_pane.object = {"error": str(e)}
        
    def __panel__(self):
        return self.layout

app = NeuroglancerViewerApp()
app.layout.servable()


Column(sizing_mode='stretch_both')
    [0] Row
        [0] Row
            [0] TextInput(placeholder='Enter Neuroglancer U..., width=650)
            [1] Button(button_type='primary', name='Load')
            [2] Button(button_type='warning', name='Demo')
    [1] Row
        [0] HTML(None, sizing_mode='stretch_width')
        [1] JSON(dict, height=600, name='Parsed URL', sizing_mode='fixed', width=400)

# V6

In [2]:
import neuroglancer

viewer_state = neuroglancer.parse_url('https://neuroglancer-demo.appspot.com/#!%7B%22dimensions%22:%7B%22x%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22y%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22z%22:%5B3.0000000000000004e-8%2C%22m%22%5D%7D%2C%22position%22:%5B5029.42333984375%2C6217.5849609375%2C1182.5%5D%2C%22crossSectionScale%22:3.7621853549999242%2C%22projectionOrientation%22:%5B-0.05179581791162491%2C-0.8017329573631287%2C0.0831851214170456%2C-0.5895944833755493%5D%2C%22projectionScale%22:4699.372698097029%2C%22layers%22:%5B%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image%22%2C%22tab%22:%22source%22%2C%22name%22:%22original-image%22%7D%2C%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image_color_corrected%22%2C%22tab%22:%22source%22%2C%22name%22:%22corrected-image%22%7D%2C%7B%22type%22:%22segmentation%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/ground_truth%22%2C%22tab%22:%22source%22%2C%22selectedAlpha%22:0.63%2C%22notSelectedAlpha%22:0.14%2C%22segments%22:%5B%223208%22%2C%224901%22%2C%2213%22%2C%224965%22%2C%224651%22%2C%222282%22%2C%223189%22%2C%223758%22%2C%2215%22%2C%224027%22%2C%223228%22%2C%22444%22%2C%223207%22%2C%223224%22%2C%223710%22%5D%2C%22name%22:%22ground_truth%22%7D%5D%2C%22layout%22:%224panel%22%7D')

viewer = neuroglancer.Viewer()
viewer

http://127.0.0.1:65514/v/944039834190b34ac559255f915287167e834bdb/

updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state
updating JSON pane
updating state

In [3]:
[print(i) for i in neuroglancer.server.global_server.viewers.items()]

('944039834190b34ac559255f915287167e834bdb', http://127.0.0.1:65514/v/944039834190b34ac559255f915287167e834bdb/)


[None]

In [4]:
viewer_state

ViewerState({"dimensions": {"x": [6.000000000000001e-09, "m"], "y": [6.000000000000001e-09, "m"], "z": [3.0000000000000004e-08, "m"]}, "position": [5029.42333984375, 6217.5849609375, 1182.5], "crossSectionScale": 3.7621853549999242, "projectionOrientation": [-0.05179581791162491, -0.8017329573631287, 0.0831851214170456, -0.5895944833755493], "projectionScale": 4699.372698097029, "layers": [{"type": "image", "source": "precomputed://gs://neuroglancer-public-data/kasthuri2011/image", "tab": "source", "name": "original-image"}, {"type": "image", "source": "precomputed://gs://neuroglancer-public-data/kasthuri2011/image_color_corrected", "tab": "source", "name": "corrected-image"}, {"type": "segmentation", "source": "precomputed://gs://neuroglancer-public-data/kasthuri2011/ground_truth", "tab": "source", "selectedAlpha": 0.63, "notSelectedAlpha": 0.14, "segments": ["3208", "4901", "13", "4965", "4651", "2282", "3189", "3758", "15", "4027", "3228", "444", "3207", "3224", "3710"], "name": "gr

In [5]:
viewer.set_state(viewer_state)

'a5f13c3e2196185d3970370c61a76c6a2659778c'

In [ ]:
def state_changed():
    print('state_changed!')

In [ ]:
viewer.shared_state.add_changed_callback(state_changed)

In [54]:
with viewer.txn() as s:
    s.voxel_coordinates = [5063, 5547, 1184]

KeyboardInterrupt: 

In [12]:
viewer.get_viewer_url()

'http://127.0.0.1:62517/v/aa0b642d3f414bb02976cc8ab8c41b32c3cb0b8d/'

In [17]:
viewer.state

ViewerState({"dimensions": {"x": [6.000000000000001e-09, "m"], "y": [6.000000000000001e-09, "m"], "z": [3.0000000000000004e-08, "m"]}, "position": [5029.42333984375, 6217.5849609375, 1182.5], "crossSectionScale": 3.7621853549999242, "projectionOrientation": [-0.05179581791162491, -0.8017329573631287, 0.0831851214170456, -0.5895944833755493], "projectionScale": 4699.372698097029, "layers": [{"type": "image", "source": "precomputed://gs://neuroglancer-public-data/kasthuri2011/image", "tab": "source", "name": "original-image"}, {"type": "image", "source": "precomputed://gs://neuroglancer-public-data/kasthuri2011/image_color_corrected", "tab": "source", "name": "corrected-image"}, {"type": "segmentation", "source": "precomputed://gs://neuroglancer-public-data/kasthuri2011/ground_truth", "tab": "source", "selectedAlpha": 0.63, "notSelectedAlpha": 0.14, "segments": ["3208", "4901", "13", "4965", "4651", "2282", "3189", "3758", "15", "4027", "3228", "444", "3207", "3224", "3710"], "name": "gr

In [63]:
import panel as pn
import neuroglancer

pn.extension()

class NeuroglancerViewerApp(pn.viewable.Viewer):
    def __init__(self, **params):
        super().__init__(**params)

        # Panel components
        self.url_input = pn.widgets.TextInput(placeholder="Enter a Neuroglancer URL and click Load", name='Input URL', width=700)
        self.load_button = pn.widgets.Button(name="Load", button_type="primary", align=('start', 'end'), width=75)
        self.load_demo_button = pn.widgets.Button(name="Demo", button_type="warning", width=75)
        self.reset_viewer_button = pn.widgets.Button(name="Stop", button_type="danger", width=75)
        self.demo_url = 'https://neuroglancer-demo.appspot.com/#!%7B%22dimensions%22:%7B%22x%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22y%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22z%22:%5B3.0000000000000004e-8%2C%22m%22%5D%7D%2C%22position%22:%5B5029.42333984375%2C6217.5849609375%2C1182.5%5D%2C%22crossSectionScale%22:3.7621853549999242%2C%22projectionOrientation%22:%5B-0.05179581791162491%2C-0.8017329573631287%2C0.0831851214170456%2C-0.5895944833755493%5D%2C%22projectionScale%22:4699.372698097029%2C%22layers%22:%5B%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image%22%2C%22tab%22:%22source%22%2C%22name%22:%22original-image%22%7D%2C%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image_color_corrected%22%2C%22tab%22:%22source%22%2C%22name%22:%22corrected-image%22%7D%2C%7B%22type%22:%22segmentation%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/ground_truth%22%2C%22tab%22:%22source%22%2C%22selectedAlpha%22:0.63%2C%22notSelectedAlpha%22:0.14%2C%22segments%22:%5B%223208%22%2C%224901%22%2C%2213%22%2C%224965%22%2C%224651%22%2C%222282%22%2C%223189%22%2C%223758%22%2C%2215%22%2C%224027%22%2C%223228%22%2C%22444%22%2C%223207%22%2C%223224%22%2C%223710%22%5D%2C%22name%22:%22ground_truth%22%7D%5D%2C%22layout%22:%224panel%22%7D'
        self.json_pane = pn.pane.JSON({}, theme='light', depth=2, name='Viewer State', height=600, width=400)
        self.shareable_url_pane = pn.pane.Markdown("**Shareable URL:**")
        self.local_url_pane = pn.pane.Markdown("**Local URL:**")
        self.iframe = pn.pane.HTML(sizing_mode='stretch_both', min_height=700, min_width=700)
        
        # setup empty viewer
        self.viewer = neuroglancer.Viewer()
        self.local_url = self.viewer.get_viewer_url()
        self.update_local_url()
        self.iframe.object = f'<iframe src="{self.local_url}" width="700" height="700"></iframe>'
        
        # Add callback to sync the viewable state with changes in viewer pane
        self.viewer.shared_state.add_changed_callback(self.update_viewable_state)

        # Add callbacks to Panel components
        self.load_button.on_click(self.update_iframe)
        self.load_demo_button.on_click(self.load_demo)
        self.reset_viewer_button.on_click(self.reset_viewer)
    
    def reset_viewer(self, event):
        neuroglancer.stop()
        self.iframe = pn.pane.HTML(sizing_mode='stretch_both', min_height=700, min_width=700)
        self.local_url_pane = pn.pane.Markdown("**Local URL:**")
        self.shareable_url_pane = pn.pane.Markdown("**Shareable URL:**")
        
    def load_demo(self, event):
        self.url_input.value = self.demo_url
        self.load_button.clicks += 1

    def update_iframe(self, event):
        new_state = neuroglancer.parse_url(self.url_input.value)
        self.viewer.set_state(new_state)
    
    def update_viewable_state(self):
        self.update_shareable_url()
        self.update_json_pane()

    def update_shareable_url(self):
        self.shareable_url = neuroglancer.to_url(self.viewer.state)
        self.shareable_url_pane.object = f"""
            <details>
                <summary><b>Shareable URL:</b></summary>
                <a href="{self.shareable_url}" target="_blank">{self.shareable_url}</a>
            </details>
        """

    def update_local_url(self):
        self.local_url_pane.object = f"""
            <details>
                <summary><b>Local URL:</b></summary>
                <a href="{self.local_url}" target="_blank">{self.local_url}</a>
            </details>
        """
            
    def update_json_pane(self):
        try:
            self.json_pane.object = self.viewer.state.to_json()
        except Exception as e:
            self.json_pane.object = {"error": str(e)}

    def __panel__(self):
        controls_layout = pn.Column(
            pn.Row(self.load_demo_button, self.load_button, self.reset_viewer_button,),
            pn.Row(self.url_input))
        links_layout = pn.Column(self.local_url_pane, self.shareable_url_pane)
        self.layout = pn.Column(
            controls_layout,
            links_layout,
            pn.Row(self.iframe, self.json_pane),
            sizing_mode='stretch_both'
        )
        return self.layout

app = NeuroglancerViewerApp()
app.servable()


Column(sizing_mode='stretch_both')
    [0] Column
        [0] Row
            [0] Button(button_type='warning', name='Demo', width=75)
            [1] Button(align=('start', 'end'), button_type='primary', name='Load', width=75)
        [1] Row
            [0] TextInput(name='Input URL', placeholder='Enter a Neuroglancer U..., width=700)
    [1] Column
        [0] Markdown(str)
        [1] Markdown(str)
    [2] Row
        [0] HTML(str, min_height=700, min_width=700, sizing_mode='stretch_both')
        [1] JSON(dict, depth=2, height=600, name='Viewer State', sizing_mode='fixed', theme='light', width=400)

In [51]:
neuroglancer.stop()

In [50]:
app.viewer

http://127.0.0.1:62517/v/8889aa34cfa1118f72f57fb4e170a4aef8f2fd18/

In [51]:
app.viewer.state

ViewerState({"dimensions": {"x": [6.000000000000001e-09, "m"], "y": [6.000000000000001e-09, "m"], "z": [3.0000000000000004e-08, "m"]}, "position": [5029.42333984375, 6217.5849609375, 1182.5], "crossSectionScale": 3.7621853549999242, "projectionOrientation": [-0.05179581791162491, -0.8017329573631287, 0.0831851214170456, -0.5895944833755493], "projectionScale": 4699.372698097029, "layers": [{"type": "image", "source": "precomputed://gs://neuroglancer-public-data/kasthuri2011/image", "tab": "source", "name": "original-image"}, {"type": "image", "source": "precomputed://gs://neuroglancer-public-data/kasthuri2011/image_color_corrected", "tab": "source", "name": "corrected-image"}, {"type": "segmentation", "source": "precomputed://gs://neuroglancer-public-data/kasthuri2011/ground_truth", "tab": "source", "selectedAlpha": 0.63, "notSelectedAlpha": 0.14, "segments": ["3208", "4901", "13", "4965", "4651", "2282", "3189", "3758", "15", "4027", "3228", "444", "3207", "3224", "3710"], "name": "gr

# V7

In [109]:
import panel as pn
import neuroglancer

pn.extension()

class NeuroglancerViewerApp(pn.viewable.Viewer):
    DEMO_URL = 'https://neuroglancer-demo.appspot.com/#!%7B%22dimensions%22:%7B%22x%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22y%22:%5B6.000000000000001e-9%2C%22m%22%5D%2C%22z%22:%5B3.0000000000000004e-8%2C%22m%22%5D%7D%2C%22position%22:%5B5029.42333984375%2C6217.5849609375%2C1182.5%5D%2C%22crossSectionScale%22:3.7621853549999242%2C%22projectionOrientation%22:%5B-0.05179581791162491%2C-0.8017329573631287%2C0.0831851214170456%2C-0.5895944833755493%5D%2C%22projectionScale%22:4699.372698097029%2C%22layers%22:%5B%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image%22%2C%22tab%22:%22source%22%2C%22name%22:%22original-image%22%7D%2C%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/image_color_corrected%22%2C%22tab%22:%22source%22%2C%22name%22:%22corrected-image%22%7D%2C%7B%22type%22:%22segmentation%22%2C%22source%22:%22precomputed://gs://neuroglancer-public-data/kasthuri2011/ground_truth%22%2C%22tab%22:%22source%22%2C%22selectedAlpha%22:0.63%2C%22notSelectedAlpha%22:0.14%2C%22segments%22:%5B%223208%22%2C%224901%22%2C%2213%22%2C%224965%22%2C%224651%22%2C%222282%22%2C%223189%22%2C%223758%22%2C%2215%22%2C%224027%22%2C%223228%22%2C%22444%22%2C%223207%22%2C%223224%22%2C%223710%22%5D%2C%22name%22:%22ground_truth%22%7D%5D%2C%22layout%22:%224panel%22%7D'

    def __init__(self, **params):
        super().__init__(**params)
        self.viewer = neuroglancer.Viewer()
        self._setup_ui_components()
        self._configure_viewer()
        self._setup_callbacks()

    def _setup_ui_components(self):
        self.url_input = pn.widgets.TextInput(
            placeholder="Enter a Neuroglancer URL and click Load", name='Input URL', width=700
        )
        self.load_button = pn.widgets.Button(name="Load", button_type="primary", width=75)
        self.demo_button = pn.widgets.Button(name="Demo", button_type="warning", width=75)
        self.json_pane = pn.pane.JSON({}, theme='light', depth=2, name='Viewer State', height=600, width=400)
        self.shareable_url_pane = pn.pane.Markdown("**Shareable URL:**")
        self.local_url_pane = pn.pane.Markdown("**Local URL:**")
        self.iframe = pn.pane.HTML(sizing_mode='stretch_both')

    def _configure_viewer(self):
        self.update_local_url()
        self.update_iframe_with_local_url()

    def _setup_callbacks(self):
        self.load_button.on_click(self._on_load_button_clicked)
        self.demo_button.on_click(self._on_demo_button_clicked)
        self.viewer.shared_state.add_changed_callback(self._on_viewer_state_changed)

    def _on_demo_button_clicked(self, event):
        self.url_input.value = self.DEMO_URL
        self._load_neuroglancer_state_from_url(self.url_input.value)

    def _on_load_button_clicked(self, event):
        self._load_neuroglancer_state_from_url(self.url_input.value)

    def _load_neuroglancer_state_from_url(self, url):
        try:
            new_state = neuroglancer.parse_url(url)
            self.viewer.set_state(new_state)
        except Exception as e:
            print(f"Error loading Neuroglancer state: {e}")

    def _on_viewer_state_changed(self):
        self.update_shareable_url()
        self.update_json_pane()

    def update_shareable_url(self):
        shareable_url = neuroglancer.to_url(self.viewer.state)
        self.shareable_url_pane.object = self._generate_details_markup("Shareable URL", shareable_url)

    def update_local_url(self):
        self.local_url_pane.object = self._generate_details_markup("Local URL", self.viewer.get_viewer_url())

    def update_iframe_with_local_url(self):
        iframe_style = 'style="width:100%; height:100%; min-width:500px; min-height:500px;"'
        self.iframe.object = f'<iframe src="{self.viewer.get_viewer_url()}" {iframe_style}"></iframe>'

    def update_json_pane(self):
        self.json_pane.object = self.viewer.state.to_json()

    def _generate_details_markup(self, title, url):
        return f"""
            <details>
                <summary><b>{title}:</b></summary>
                <a href="{url}" target="_blank">{url}</a>
            </details>
        """

    def __panel__(self):
        controls_layout = pn.Column(
            pn.Row(self.demo_button, self.load_button),
            pn.Row(self.url_input))
        links_layout = pn.Column(self.local_url_pane, self.shareable_url_pane)
        return pn.Column(
            controls_layout,
            links_layout,
            pn.FlexBox(self.iframe, pn.Card(self.json_pane, title='App State', collapsed=True)))

app = NeuroglancerViewerApp()
app.servable()


Column
    [0] Column
        [0] Row
            [0] Button(button_type='warning', name='Demo', width=75)
            [1] Button(button_type='primary', name='Load', width=75)
        [1] Row
            [0] TextInput(name='Input URL', placeholder='Enter a Neuroglancer U..., width=700)
    [1] Column
        [0] Markdown(str)
        [1] Markdown(str)
    [2] FlexBox(objects=[HTML(str, sizing_mode='st...], sizing_mode='stretch_width')

In [84]:
with app.viewer.txn() as s:
    s.voxel_coordinates = [2689, 8465, 1200]

In [85]:
with app.viewer.txn() as s:
    s.layers['segmentation'].visible = False